In [53]:
import numpy as np

N = 5
# 比特数2^N
n_qubits = N**2

# 距离矩阵
D = np.array([[0,  13, 11, 15, 8],
              [13, 0,  7,  14, 9],
              [11, 7,  0,  10, 9],
              [15, 14, 10, 0,  12],
              [8,  9,  9,  12, 0]])

# 约束权重
lambda_val = np.max(D)

# 初始化QUBO矩阵
Q = np.zeros((n_qubits, n_qubits))

# 填充路径代价项
for t in range(N-1):
    for c1 in range(N):
        for c2 in range(N):
            Q[N*t + c1, N*(t+1) + c2] += D[c1, c2]

# 添加约束项
for t in range(N):
    for c1 in range(N):
        for c2 in range(N):
            Q[N*t + c1, N*t + c2] += lambda_val * (-1 if c1 == c2 else 1)

for c in range(N):
    for t1 in range(N):
        for t2 in range(N):
            Q[N*t1 + c, N*t2 + c] += lambda_val * (-1 if t1 == t2 else 1)

In [54]:
import openjij as oj

# 转换为 OpenJij 的 BinaryQuadraticModel
bqm = oj.BinaryQuadraticModel.from_numpy_matrix(
    Q, 
    vartype='BINARY'  # 变量类型：二值 (0或1)
)

In [55]:
# 创建模拟退火采样器
sampler = oj.SASampler()

# 执行退火（默认参数）
sampleset = sampler.sample(bqm, num_reads=20)

# 查看结果
print(sampleset)

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 ... 24 energy num_oc.
0   1  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  1 ...  0 -116.0       1
5   1  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  1 ...  0 -116.0       1
18  1  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  1 ...  0 -116.0       1
19  1  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  1 ...  0 -116.0       1
2   0  1  0  0  0  0  0  1  0  0  0  0  0  1  0  0  0  0 ...  0 -113.0       1
8   1  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  1 ...  0 -113.0       1
4   0  0  0  1  0  0  1  0  0  0  0  0  1  0  0  0  0  0 ...  0 -112.0       1
9   0  1  0  0  0  0  0  0  0  1  1  0  0  0  0  0  0  1 ...  0 -112.0       1
11  0  0  0  1  0  0  0  1  0  0  1  0  0  0  0  0  0  0 ...  0 -112.0       1
13  1  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  1  0 ...  0 -112.0       1
6   0  0  0  1  0  1  0  0  0  0  0  0  0  0  1  0  0  1 ...  0 -111.0       1
12  0  0  0  1  0  0  0  0  0  1  0  1  0  0  0  0  

In [56]:
best_solution = sampleset.first.sample

# 解码函数
def decode_tsp(solution):
    cities = ['A', 'B', 'C', 'D', 'E']
    path = [''] * N
    for idx, val in solution.items():
        if val == 1:
            time_step = idx % N
            city_idx = idx // N
            path[time_step] = cities[city_idx]
    return '->'.join(path) if all(path) else "无效路径"

# 验证并输出
decoded_path = decode_tsp(best_solution)
if len(set(decoded_path.split('->'))) == N:
    print("最优路径:", decoded_path)
else:
    print("未找到合法路径，请检查QUBO约束权重")
    print("当前激活比特:", [k for k, v in best_solution.items() if v == 1])

最优路径: A->C->D->E->B
